In [ ]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from descriptors import preprocess_image, extract_descriptor, extract_descriptors

# Paths
IMG_FOLDER = "../Data/Week3/qst1_w3/"
IMG_FOLDER_GT = "../Data/Week3/BBDD/"
DESC_GT_PATH = "results/descriptors_gt.pkl"
RESULTS_PATH = "results/query_to_gt_top10.pkl"

K = 10  # number of best matches to retrieve

# --- 1. Load or compute GT descriptors ---
if os.path.exists(DESC_GT_PATH):
    print(f"✅ Loading GT descriptors from {DESC_GT_PATH}")
    with open(DESC_GT_PATH, "rb") as f:
        data = pickle.load(f)
        desc_gt = data["desc_gt"]
        gt_names = data["gt_names"]
else:
    print("🧠 Computing GT descriptors...")
    desc_gt, gt_names = extract_descriptors(IMG_FOLDER_GT, preprocess=False)
    
    # Save to file for later use
    os.makedirs(os.path.dirname(DESC_GT_PATH), exist_ok=True)
    with open(DESC_GT_PATH, "wb") as f:
        pickle.dump({"desc_gt": desc_gt, "gt_names": gt_names}, f)
    print(f"💾 Saved GT descriptors to {DESC_GT_PATH}")

# --- 2. Process query images ---
image_names = sorted([f for f in os.listdir(IMG_FOLDER) if f.endswith('.jpg')])
results = []

for img_name in image_names:
    print(f"\nProcessing {img_name} ...")
    img_path = os.path.join(IMG_FOLDER, img_name)
    img = cv2.imread(img_path)

    if img is None:
        print(f"⚠️ Skipping {img_name}: could not read image.")
        continue

    # Preprocess and extract descriptor
    img = preprocess_image(img)
    desc = extract_descriptor(img)

    # --- 3. Compare with GT descriptors using cosine similarity ---
    similarities = cosine_similarity([desc], desc_gt)[0]
    
    # Get indices of top-K most similar GT images
    top_k_indices = np.argsort(-similarities)[:K]  # sort descending
    results.append(top_k_indices.tolist())

# --- 4. Save results ---
os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
with open(RESULTS_PATH, "wb") as f:
    pickle.dump(results, f)

print(f"\n✅ Saved top-{K} retrieval results to {RESULTS_PATH}")


In [ ]:
print(results)